# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119889e+02     1.658846e+00
 * time: 0.07575297355651855
     1     1.060450e+01     9.632346e-01
 * time: 1.2338900566101074
     2    -1.142914e+01     1.157506e+00
 * time: 1.3780109882354736
     3    -3.349209e+01     8.629533e-01
 * time: 1.4997010231018066
     4    -4.686964e+01     6.225366e-01
 * time: 1.6339268684387207
     5    -5.677751e+01     2.202509e-01
 * time: 1.7729899883270264
     6    -5.971688e+01     1.835601e-01
 * time: 1.8882949352264404
     7    -6.093897e+01     6.969099e-02
 * time: 1.9825739860534668
     8    -6.145500e+01     4.801102e-02
 * time: 2.07785701751709
     9    -6.174757e+01     3.224388e-02
 * time: 2.1731550693511963
    10    -6.190741e+01     2.431138e-02
 * time: 2.269676923751831
    11    -6.200759e+01     2.044556e-02
 * time: 2.3760600090026855
    12    -6.206729e+01     1.867479e-02
 * time: 2.469666004180908
    13    -6.211439e+01     1.387622e-02
 * time: 2.5645048618316

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557681
    AtomicNonlocal      14.8522650
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336820

    total               -62.261666461889
